In [ ]:
# @title Packages install {"vertical-output":true,"display-mode":"form"}

install.packages("caret")
install.packages("lubridate")
install.packages("tidyverse")

library(tidyverse)
library(lubridate)
library(caret)

In [ ]:
# @title Dataset Reading {"vertical-output":true,"display-mode":"form"}

data <- read.csv("train.csv")
head(data)
summary(data)

In [ ]:
# @title Dataset Filter By Product and Store {"vertical-output":true,"display-mode":"form"}

data <- data %>% filter(store == 2 & product == 3)
head(data)
summary(data)

In [ ]:
# @title Split Date field {"vertical-output":true,"display-mode":"form"}

data$Date <- as.Date(data$Date, format = "%Y-%m-%d")

data <- data %>%
  mutate(
    Year = year(Date),
    Month = as.integer(month(Date)),
    DayOfWeek = as.integer(wday(Date, label = TRUE)),
    DayOfMonth = as.integer(day(Date))
  )

head(data)

In [ ]:
# @title Date One-Hot encoding {"vertical-output":true,"display-mode":"form"}

#dummies <- dummyVars(~ Month + DayOfWeek + DayOfMonth, data = data)
#data_transformed <- predict(dummies, newdata = data)
#data <- cbind(data, data_transformed)
#head(data)

In [ ]:
# @title Add Lag columns {"vertical-output":true,"display-mode":"form"}

#data <- data %>%
#  mutate(
#    Lag1 = lag(number_sold, 1),
#    Lag2 = lag(number_sold, 2)
#  )
#head(data)

In [ ]:
# @title Lags scaling {"vertical-output":true,"display-mode":"form"}

#min_val = min(data$number_sold)
#max_val = max(data$number_sold)
#data$Lag1 <- sapply(data$Lag1, function(x) (x - min_val) / (max_val - min_val))
#data$Lag2 <- sapply(data$Lag2, function(x) (x - min_val) / (max_val - min_val))
#
#head(data)

In [ ]:
# @title Time Chart {"vertical-output":true,"display-mode":"form"}

chart_data <- data %>% filter(DayOfMonth == 1)

ggplot() +
  geom_line(data=chart_data, aes(x = 1:nrow(chart_data), y = number_sold), color = "blue") +
  labs(title = "Ventas mensuales por tienda y producto desde 2010 hasta 2018", x = "Semana", y = "Ventas") +
  theme_minimal()

chart_data_2010 <- data %>% filter(DayOfMonth %% 7 == 0 & Year == 2010)

ggplot() +
  geom_line(data=chart_data_2010, aes(x = 1:nrow(chart_data_2010), y = number_sold), color = "blue") +
  labs(title = "Ventas semanales por tienda y producto en 2010", x = "Semana", y = "Ventas") +
  theme_minimal()

chart_data_2013 <- data %>% filter(DayOfMonth %% 7 == 0 & Year == 2013)

ggplot() +
  geom_line(data=chart_data_2013, aes(x = 1:nrow(chart_data_2013), y = number_sold), color = "blue") +
  labs(title = "Ventas semanales por tienda y producto en 2013", x = "Semana", y = "Ventas") +
  theme_minimal()

chart_data_2016 <- data %>% filter(DayOfMonth %% 7 == 0 & Year == 2016)

ggplot() +
  geom_line(data=chart_data_2016, aes(x = 1:nrow(chart_data_2016), y = number_sold), color = "blue") +
  labs(title = "Ventas semanales por tienda y producto en 2016", x = "Semana", y = "Ventas") +
  theme_minimal()

chart_data_2018 <- data %>% filter(DayOfMonth %% 7 == 0 & Year == 2018)

ggplot() +
  geom_line(data=chart_data_2018, aes(x = 1:nrow(chart_data_2018), y = number_sold), color = "blue") +
  labs(title = "Ventas semanales por tienda y producto en 2018", x = "Semana", y = "Ventas") +
  theme_minimal()

In [ ]:
# @title Linear Regression {"vertical-output":true,"display-mode":"form"}

lin_reg <- train(number_sold~Date, data=data, method="lm",metric="RMSE")
print(summary(lin_reg))

chart_data <- data %>% filter(DayOfMonth == 1)

ggplot() +
  geom_line(data=chart_data, aes(x = 1:nrow(chart_data), y = number_sold), color = "blue") +
  geom_line(data=chart_data, aes(x = 1:nrow(chart_data), y = predict(lin_reg, chart_data)),color = "red")+
  labs(title = "Regresión Lineal", x = "X", y = "y") +
  theme_minimal()

In [ ]:
# @title K-Nearest Neighbors {"vertical-output":true,"display-mode":"form"}

knn_reg <- train(number_sold~Date, data=data, method="knn",metric="RMSE")
print(summary(knn_reg))

chart_data <- data %>% filter(DayOfMonth == 1)

ggplot() +
  geom_line(data=chart_data, aes(x = 1:nrow(chart_data), y = number_sold), color = "blue") +
  geom_line(data=chart_data, aes(x = 1:nrow(chart_data), y = predict(knn_reg, chart_data)),color = "red")+
  labs(title = "KNN", x = "X", y = "y") +
  theme_minimal()

In [ ]:
# @title Tree {"vertical-output":true,"display-mode":"form"}

tree_reg <- train(number_sold~Date, data=data, method="rpart",metric="RMSE")

chart_data <- data %>% filter(DayOfMonth == 1)

ggplot() +
  geom_line(data=chart_data, aes(x = 1:nrow(chart_data), y = number_sold), color = "blue") +
  geom_line(data=chart_data, aes(x = 1:nrow(chart_data), y = predict(tree_reg, chart_data)),color = "red")+
  labs(title = "Tree", x = "X", y = "y") +
  theme_minimal()

**Conclusiones**

Ninguno de los 3 algortimos manejan correctamente una serie de tiempo.
Las series de tiempo tienen un comportamiento bastante particular, tienen una serie de patrones propios que deben descomponerse y analizarse por separado.
Las series de tiempo tienen un espacio propio dentro de la ciencia de datos, no es una simple aplicación sino que es, por sí misma, un área de estudio.
Las series de tiempo tienen comportamientos que siguen patrones fuertemente cíclicos que no son manejados correctamente por modelos que no están específicamente preparado para esto:
* Regresión lineal: solamente es capaz de detectar la tendencia general y no fluctuaciones repetitivas. Asume la función dependendiente será infinitamente grande o pequeña en el tiempo, por las propias características de una función lineal.
* KNN: dentro del rango de tiempo evaluado, genera resultados muy ajustados a la muestra. Fuera del mismo, no tiene ningún tipo de capacidad, las mismas K muestras genrarán un valor fijo hasta el tiempo infinito. Genera un comportamiento escalonado que no responde a la conducta de una variable en series de tiempo, cada escalón es un promedio de K muestra cercanas.
* Árbol: similar al algorimo KNN, genera una serie de escalones en donde se asume que los X vecinos van a tener el mismo valor. Asume que los X cercanos tienen exactamente el mismo comportamiento.

Los 3 modelos presentados no son aptos para manejar series de tiempo pero tampoco son defectuosos, simplemente tienen otro tipo de aplicaciones.




